In [ ]:
import pandas as pd
import numpy as np
import pickle

import datetime
import pymongo
from pymongo import MongoClient

import paho.mqtt.client as mqtt
import time
import warnings
warnings.filterwarnings('ignore')
pred = 0
def get_model():
    """This function returns a Machine Learning model 
    trained to predict Malaria transmission period 
    based on climate data variability"""
    with open('model_pickle','rb') as f:
        model = pickle.load(f)
    return model

def mqtt_connect():
    """This function help to connect to local MQTT 
    (Message Queing Telemetry Transport protocol) Broker"""
    client = mqtt.Client()
    client.connect("127.0.0.1",1883)
    return client

class MalariaData():
    """This class return temperature, humidity,
    rainfall and date data 
    from the MALARIA database hosted by mongoDB"""
    def __init__(self,temp = 0,humidity = 0,rain = 0,date = datetime.date.today()):
        self.temp = temp
        self.hum = humidity
        self.rain = rain
        self.date =date
        
    def db_connection(self):
        connection = MongoClient('localhost',27017)
        db = connection.MALARIA
        return db
    
    def get_temp(self,db):
        table = db.Sensors_Temp
        Temp = table.find()
        df = pd.DataFrame(list(Temp))
        del df['_id']
        self.temp = df
        return self.temp
    
    def get_hum(self,db):
        table = db.Sensors_Hum
        Hum = table.find()
        df = pd.DataFrame(list(Hum))
        del df['_id']
        self.hum = df 
        return self.hum
    
    def get_date(self,db):
        table = db.DATE
        DATE = table.find()
        df=pd.DataFrame(list(DATE))
        del df['_id']
        df['DATE']=df.DATE.apply(lambda d:str(d.year)+'-'+str(d.month)+'-'+str(d.day))
        self.date = df 
        return self.date
    
    def get_rain(self,db):
        table = db.Sensors_Rain
        Rain = table.find()
        df = pd.DataFrame(list(Rain))
        del df['_id']
        self.Rain = df 
        return self.Rain
DATA=MalariaData()

def calculate_mean(date_data,temp_data,hum_data):
    """This function calculate temperature mean 
    and humidity mean 
    from the data generated by
    the MalariaData class and return temp mean and humidity mean"""
    
    data=pd.merge(date_data,temp_data, how='outer', left_index=True, right_index=True)
    data=pd.merge(data,hum_data, how='outer', left_index=True, right_index=True)
    data.to_csv(r'C:\Users\nijas2012\Desktop\ACEIoT\climate_data.csv')
    now = datetime.datetime.now()
    delta = datetime.timedelta(days = now.day) 
    today=datetime.date.today()
    start_time=str(today - delta) 
    data=data[data['DATE']>=start_time]

    Temp=data.iloc[:,1].astype('f')
    mean_temp=Temp.mean()

    Hum=data.iloc[:,2].astype('f')
    mean_hum=Hum.mean()
    
    return mean_temp,mean_hum
    
def Malaria_status_publish(mqttclient,pred):
    if(pred==2): 
        
        mqttclient.publish("ACEIoT/janvier/malaria/prediction" ,"Ab-normal")
        mqttclient.publish("ACEIoT/janvier/malaria/date" ,str(datetime.date.today()))
        mqttclient.publish("ACEIoT/janvier/malaria/time" ,str(now.hour)+':'+str(now.minute)+':'+str(now.second)+str('AM' if (now.hour <=12)else 'PM'))
        time.sleep(8)
        mqttclient.publish("ACEIoT/janvier/malaria/prediction" ,"")
        mqttclient.publish("ACEIoT/janvier/malaria/date" ,"")
        mqttclient.publish("ACEIoT/janvier/malaria/time" ,"")
        time.sleep(0.5)

    else:
        
        mqttclient.publish("ACEIoT/janvier/malaria/prediction" ,"Normal")
        mqttclient.publish("ACEIoT/janvier/malaria/date" ,str(datetime.date.today()))
        mqttclient.publish("ACEIoT/janvier/malaria/time" ,str(now.hour)+':'+str(now.minute)+':'+str(now.second)+str('AM' if (now.hour <=12)else 'PM'))
        time.sleep(8)
        mqttclient.publish("ACEIoT/janvier/malaria/prediction" ,"")
        mqttclient.publish("ACEIoT/janvier/malaria/date" ,"")
        mqttclient.publish("ACEIoT/janvier/malaria/time" ,"")
        time.sleep(0.5)

def publish_data(temp,hum,rain,mqttclient):
    """This function publish temperature,
    humidity and rainfall data 
    from database at MQTT Broker"""
    temp=temp.astype('f')
    T=temp['TEMPERATURE'][len(temp)-1]
    hum=hum.astype('f')
    H=hum['HUMIDITY'][len(hum)-1]
    rain=rain.astype('f')
    R=rain['RAINFALL'][len(rain)-1]
    
    mqttclient.publish("ACEIoT/janvier/temperature/db" ,str(T)) 
    mqttclient.publish("ACEIoT/janvier/humidity/db" ,str(H)) 
    mqttclient.publish("ACEIoT/janvier/rain/db" ,str(R)) 
    
    
while(True):

    publish_data(DATA.get_temp(DATA.db_connection()),
                 DATA.get_hum(DATA.db_connection()),
                 DATA.get_rain(DATA.db_connection()),
                 mqtt_connect()
                )
    model = get_model()
    now = datetime.datetime.now()
    month_list1 = [1,3,5,7,8,10,12]
    month_list2 = [2,4,6,9,11]

    if now.month in month_list1:
        if(now.day==31):
            data = calculate_mean(DATA.get_date(TEMP.db_connection()),
                                  DATA.get_temp(TEMP.db_connection()),
                                  DATA.get_hum(TEMP.db_connection())
                                )
            pred = model.predict([[data[0],data[0],data[1],data[1]]])
        Malaria_status_publish(mqtt_connect(),pred)

    elif now.month in month_list2:

        if(now.day==30):
            data = calculate_mean(DATA.get_date(TEMP.db_connection()),
                                  DATA.get_temp(TEMP.db_connection()),
                                  DATA.get_hum(TEMP.db_connection())
                                 )
            pred = model.predict([[data[0],data[0],data[1],data[1]]])
        Malaria_status_publish(mqtt_connect(),pred)
    else:
        if(now.day==28 or now.day==29):
            data = calculate_mean(DATA.get_date(TEMP.db_connection()),
                                  DATA.get_temp(TEMP.db_connection()),
                                  DATA.get_hum(TEMP.db_connection())
                                 )

            pred = model.predict([[data[0],data[0],data[1],data[1]]])
        Malaria_status_publish(mqtt_connect(),pred)

In [2]:
!pip install pymongo

  Using cached pymongo-3.11.0-cp38-cp38-win_amd64.whl (382 kB)


In [3]:
!pip install paho-mqtt

Processing c:\users\nijas\appdata\local\pip\cache\wheels\c6\63\e1\6e3a42c72eb48428f83a5718662fc2273b0ffe7f644085cc4e\paho_mqtt-1.5.0-py3-none-any.whl


In [8]:
!pip install -U scikit-learn